<a href="https://colab.research.google.com/github/kartal0482/botlarim/blob/main/cex_arbitraj_botu_14borsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install ccxt rich --quiet

from rich.console import Console
from rich.live import Live
from rich.table import Table
from rich.panel import Panel
from collections import Counter
import ccxt
import json
import os
import time

console = Console()

exchanges = {
    "KuCoin": ccxt.kucoin(),
    "MEXC": ccxt.mexc(),
    "OKX": ccxt.okx(),
    "Phemex": ccxt.phemex(),
    "Gate": ccxt.gateio(),
    "Bitget": ccxt.bitget(),
    "WhiteBit": ccxt.whitebit(),
    "Bitrue": ccxt.bitrue(),
    "Huobi": ccxt.huobi(),
    "BitMart": ccxt.bitmart(),
    "LBank": ccxt.lbank(),
    "BingX": ccxt.bingx(),
    "XT": ccxt.xt(),
}

exclude_coins = {
    "USDT", "USDC", "BUSD", "DAI", "WBTC", "WETH",
    "UNI", "AAVE", "PEPE", "PI", "OKB", "TAO", "APT",
    "NEAR", "ICP", "ETC", "CRO", "ONDO", "USD1", "KAS",
    "POL", "MNT", "GT", "BONK", "VET", "TRUMP", "SKY",
    "RENDER", "ARB", "ENA"
}

price_cache = {}
price_cache_time = {}
CACHE_EXPIRY = 10  # saniye

previous_pct_map = {}

def normalize_pair(pair):
    return pair.replace("_", "/")

def is_cache_outdated(cache_file, hours=24):
    if not os.path.exists(cache_file):
        return True
    modified_time = os.path.getmtime(cache_file)
    age_hours = (time.time() - modified_time) / 3600
    return age_hours > hours

def load_or_get_common_pairs():
    cache_file = "ortak_pariteler.json"
    if os.path.exists(cache_file) and not is_cache_outdated(cache_file):
        try:
            with open(cache_file, "r") as f:
                return json.load(f)
        except:
            pass

    all_sets = []
    for name, ex in exchanges.items():
        try:
            markets = ex.load_markets()
            if name == "LBank":
                pairs = set(normalize_pair(p) for p in markets if p.endswith('_USDT'))
            else:
                pairs = set(p for p in markets if p.endswith('/USDT'))
            all_sets.append(pairs)
        except:
            continue

    all_pairs = []
    for s in all_sets:
        all_pairs += list(s)
    counter = Counter(all_pairs)
    common_pairs = [p for p, c in counter.items() if c >= 2]
    filtered_pairs = [p for p in common_pairs if p.split("/")[0] not in exclude_coins]

    with open(cache_file, "w") as f:
        json.dump(filtered_pairs, f)

    return filtered_pairs

def get_bid_ask_price(exchange, pair, name):
    key = (name, pair)
    now = time.time()
    if key in price_cache and (now - price_cache_time[key] < CACHE_EXPIRY):
        return price_cache[key]

    try:
        p = pair.replace("/", "_") if name == "LBank" else pair
        ticker = exchange.fetch_ticker(p)
        price_cache[key] = {
            "ask": ticker.get("ask"),
            "bid": ticker.get("bid")
        }
        price_cache_time[key] = now
        return price_cache[key]
    except:
        return None

def get_withdraw_status(exchange, symbol):
    try:
        info = exchange.fetch_currencies()
        return info.get(symbol, {}).get('withdraw', "Belirsiz")
    except:
        return "Belirsiz"

def get_deposit_status(exchange, symbol):
    try:
        info = exchange.fetch_currencies()
        return info.get(symbol, {}).get('deposit', "Belirsiz")
    except:
        return "Belirsiz"

def build_table(data):
    table = Table(show_header=True, show_edge=False, box=None, padding=(0, 0), expand=False)
    table.add_column("No", justify="left", style="grey70", width=3)
    table.add_column("Parite", style="bold cyan", width=15)
    table.add_column("% Kar", justify="right", style="bold", width=10)
    table.add_column("Alış (Borsa)", justify="left", width=38)
    table.add_column("Satış (Borsa)", justify="left", width=38)

    for i, row in enumerate(data, 1):
        parite = row["Parite"]
        yuzde = row["Yuzde"]

        onceki = previous_pct_map.get(parite)
        if onceki is None:
            ok = ""
        elif yuzde > onceki:
            ok = "[green]↑[/green]"
        elif yuzde < onceki:
            ok = "[red]↓[/red]"
        else:
            ok = ""
        previous_pct_map[parite] = yuzde

        alis_cekme = f"çekme:{'✅' if row['AlisCekme'] == True else '❌' if row['AlisCekme'] == False else 'Belirsiz'}"
        satis_yatirma = f"yatırma:{'✅' if row['SatisYatirma'] == True else '❌' if row['SatisYatirma'] == False else 'Belirsiz'}"
        alis = f"{row['AlisFiyat']:.6f} ({row['AlisBorsa']} - {alis_cekme})"
        satis = f"{row['SatisFiyat']:.6f} ({row['SatisBorsa']} - {satis_yatirma})"

        table.add_row(
            str(i),
            parite,
            f"{yuzde:.2f}% {ok}",
            alis,
            satis,
        )
        table.add_section()
    return table

def main():
    console.log("Ortak pariteler dosyadan yükleniyor...")
    common_pairs = load_or_get_common_pairs()
    console.log(f"{len(common_pairs)} ortak parite yüklendi.")
    symbols = common_pairs[:60]

    with Live(console=console, refresh_per_second=3, screen=False) as live:
        while True:
            all_rows = []
            for pair in symbols:
                price_data = {}
                for name, ex in exchanges.items():
                    ticker = get_bid_ask_price(ex, pair, name)
                    if ticker and ticker["ask"] and ticker["bid"]:
                        price_data[name] = ticker
                if len(price_data) < 2:
                    continue

                alis = min(price_data.items(), key=lambda x: x[1]["ask"])
                satis = max(price_data.items(), key=lambda x: x[1]["bid"])
                pct = ((satis[1]["bid"] - alis[1]["ask"]) / alis[1]["ask"]) * 100

                if pct < 0.5 or pct > 50:
                    continue

                all_rows.append({
                    "Parite": pair,
                    "AlisFiyat": alis[1]["ask"],
                    "AlisBorsa": alis[0],
                    "SatisFiyat": satis[1]["bid"],
                    "SatisBorsa": satis[0],
                    "Yuzde": pct,
                })

            top_rows = sorted(all_rows, key=lambda x: x["Yuzde"], reverse=True)[:20]
            final_rows = []

            for row in top_rows:
                coin = row["Parite"].split("/")[0]
                withdraw_ok = get_withdraw_status(exchanges[row["AlisBorsa"]], coin)
                deposit_ok = get_deposit_status(exchanges[row["SatisBorsa"]], coin)
                if withdraw_ok is True and deposit_ok is True:
                    row["AlisCekme"] = withdraw_ok
                    row["SatisYatirma"] = deposit_ok
                    final_rows.append(row)

            table = build_table(final_rows)
            live.update(Panel(table, title="✅ Çekme & Yatırma Açık İlk 20 Arbitraj (%0.5 - %50)", border_style="cyan"))
            time.sleep(10)

if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.0/628.0 kB 45.0 MB/s eta 0:00:00


[17:31:21] Ortak pariteler dosyadan yükleniyor...                                 ]8;id=688796;file:///tmp/ipython-input-1-3858627697.py\ipython-input-1-3858627697.py]8;;\:]8;id=950770;file:///tmp/ipython-input-1-3858627697.py#159\159]8;;\

[17:31:53] 2657 ortak parite yüklendi.                                            ]8;id=801261;file:///tmp/ipython-input-1-3858627697.py\ipython-input-1-3858627697.py]8;;\:]8;id=860319;file:///tmp/ipython-input-1-3858627697.py#161\161]8;;\

Output()